# Open AI Modellerini İnce Ayarlama

Bu not defteri, Open AI'nin [İnce Ayarlama](https://platform.openai.com/docs/guides/fine-tuning?WT.mc_id=academic-105485-koreyst) dokümantasyonunda sunulan güncel rehberliğe dayanmaktadır.

İnce ayarlama, temel modellerin performansını uygulamanız için, o belirli kullanım senaryosuna veya bağlama uygun ek veri ve içerikle yeniden eğiterek artırır. _Az örnekli öğrenme_ ve _geri getirme ile zenginleştirilmiş üretim_ gibi istem mühendisliği tekniklerinin, varsayılan istemi ilgili verilerle geliştirerek kaliteyi artırmanıza olanak tanıdığını unutmayın. Ancak, bu yaklaşımlar hedeflenen temel modelin maksimum token pencere boyutuyla sınırlıdır.

İnce ayarlama ile, modeli ihtiyaç duyulan verilerle doğrudan yeniden eğitmiş oluruz (maksimum token penceresine sığabilecekten çok daha fazla örnek kullanmamıza olanak tanır) ve artık çıkarım sırasında örnek verilmesine gerek olmayan _özel_ bir model sürümü dağıtmış oluruz. Bu, sadece istem tasarımımızın etkinliğini artırmakla kalmaz (token penceresini başka şeyler için kullanmakta daha esnek oluruz), aynı zamanda maliyetlerimizi de potansiyel olarak azaltır (çıkarım sırasında modele göndermemiz gereken token sayısını azaltarak).

İnce ayarlama 4 adımdan oluşur:
1. Eğitim verisini hazırlayın ve yükleyin.
1. İnce ayarlama eğitimi başlatın ve ince ayarlanmış modeli elde edin.
1. İnce ayarlanmış modeli değerlendirin ve kalite için yineleyin.
1. Memnun kaldığınızda, ince ayarlanmış modeli çıkarım için dağıtın.

Tüm temel modellerin ince ayarlamayı desteklemediğini unutmayın - en güncel bilgi için [OpenAI dokümantasyonunu kontrol edin](https://platform.openai.com/docs/guides/fine-tuning/what-models-can-be-fine-tuned?WT.mc_id=academic-105485-koreyst). Daha önce ince ayarlanmış bir modeli de tekrar ince ayarlayabilirsiniz. Bu eğitimde, ince ayarlama için hedef temel model olarak `gpt-35-turbo` kullanacağız.

---


### Adım 1.1: Veri Setinizi Hazırlayın

Periyodik tabloyu anlamanıza yardımcı olacak ve bir element hakkında sorulan sorulara limerick tarzında cevap veren bir sohbet botu oluşturalım. _Bu_ basit eğitimde, modelimizi eğitmek için verinin beklenen formatını gösteren birkaç örnek yanıt içeren bir veri seti oluşturacağız. Gerçek bir kullanım senaryosunda, çok daha fazla örnek içeren bir veri seti hazırlamanız gerekir. Ayrıca, varsa, uygulama alanınıza uygun açık bir veri setini de kullanabilir ve ince ayar için yeniden biçimlendirebilirsiniz.

`gpt-35-turbo` üzerine odaklandığımız ve tek seferlik bir yanıt (sohbet tamamlaması) istediğimiz için, [önerilen bu formatı](https://platform.openai.com/docs/guides/fine-tuning/preparing-your-dataset?WT.mc_id=academic-105485-koreyst) kullanarak OpenAI sohbet tamamlaması gereksinimlerini yansıtan örnekler oluşturabiliriz. Eğer çok adımlı (çok turlu) bir sohbet içeriği bekliyorsanız, [çok turlu örnek formatını](https://platform.openai.com/docs/guides/fine-tuning/multi-turn-chat-examples?WT.mc_id=academic-105485-koreyst) kullanmalısınız; bu formatta, ince ayar sürecinde hangi mesajların kullanılıp kullanılmayacağını belirten bir `weight` parametresi bulunur.

Burada eğitimimiz için daha basit olan tek turlu formatı kullanacağız. Veriler [jsonl formatında](https://jsonlines.org/?WT.mc_id=academic-105485-koreyst) olacak; her satırda bir kayıt bulunacak ve her biri JSON formatında bir nesne olarak temsil edilecek. Aşağıdaki kod parçası, örnek olarak 2 kayıt göstermektedir – tam örnek seti (10 örnek) için [training-data.jsonl](../../../../../18-fine-tuning/python/openai/training-data.jsonl) dosyasına bakabilirsiniz; bu dosyayı ince ayar eğitimimizde kullanacağız. **Not:** Her kayıt _mutlaka_ tek bir satırda tanımlanmalıdır (biçimlendirilmiş bir JSON dosyasında olduğu gibi satırlara bölünmemelidir)

```json
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Gallium"}, {"role": "assistant", "content": "Gallium, oh gallium, so light - Melts in your hand, oh what a sight - At 86 degrees - Its liquid with ease - And in semiconductors, it's out of sight"}]}
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Hydrogen"}, {"role": "assistant", "content": "Hydrogen, the first in the line - The lightest of all, so divine - It's in water, you see - And in stars, it's the key - The universe's most common sign"}]}
```

Gerçek bir kullanım senaryosunda, iyi sonuçlar almak için çok daha büyük bir örnek setine ihtiyacınız olacak – burada yanıt kalitesi ile ince ayar için harcanacak zaman/maliyet arasında bir denge kurmanız gerekecek. Biz süreci göstermek için küçük bir set kullanıyoruz, böylece ince ayarı hızlıca tamamlayabiliriz. Daha karmaşık bir ince ayar eğitimi için [bu OpenAI Cookbook örneğine](https://github.com/openai/openai-cookbook/blob/main/examples/How_to_finetune_chat_models.ipynb?WT.mc_id=academic-105485-koreyst) göz atabilirsiniz.


### Adım 1.2 Verisetinizi Yükleyin

Verinizi, Dosyalar API'sini kullanarak [burada açıklandığı gibi](https://platform.openai.com/docs/guides/fine-tuning/upload-a-training-file) yükleyin. Bu kodu çalıştırabilmek için öncelikle aşağıdaki adımları tamamlamış olmanız gerekir:
 - `openai` Python paketini yüklediniz (en son özellikler için >=0.28.0 sürümünü kullandığınızdan emin olun)
 - `OPENAI_API_KEY` ortam değişkenini OpenAI API anahtarınızla ayarladınız
Daha fazla bilgi için, kurs kapsamında sunulan [Kurulum rehberine](./../../../00-course-setup/02-setup-local.md?WT.mc_id=academic-105485-koreyst) göz atabilirsiniz.

Şimdi, yerel JSONL dosyanızdan yükleme için bir dosya oluşturmak üzere kodu çalıştırın.


In [24]:
from openai import OpenAI
client = OpenAI()

ft_file = client.files.create(
  file=open("./training-data.jsonl", "rb"),
  purpose="fine-tune"
)

print(ft_file)
print("Training File ID: " + ft_file.id)

FileObject(id='file-JdAJcagdOTG6ACNlFWzuzmyV', bytes=4021, created_at=1715566183, filename='training-data.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)
Training File ID: file-JdAJcagdOTG6ACNlFWzuzmyV


### Adım 2.1: SDK ile İnce Ayar (Fine-tuning) İşini Oluşturun


In [25]:
from openai import OpenAI
client = OpenAI()

ft_filejob = client.fine_tuning.jobs.create(
  training_file=ft_file.id, 
  model="gpt-3.5-turbo"
)

print(ft_filejob)
print("Fine-tuning Job ID: " + ft_filejob.id)

FineTuningJob(id='ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', created_at=1715566184, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-EZ6ag0n0S6Zm8eV9BSWKmE6l', result_files=[], seed=830529052, status='validating_files', trained_tokens=None, training_file='file-JdAJcagdOTG6ACNlFWzuzmyV', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)
Fine-tuning Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh


### Adım 2.2: İşin Durumunu Kontrol Et

`client.fine_tuning.jobs` API'si ile yapabileceğiniz birkaç şey şunlardır:
- `client.fine_tuning.jobs.list(limit=<n>)` - Son n ince ayar işini listele
- `client.fine_tuning.jobs.retrieve(<job_id>)` - Belirli bir ince ayar işinin detaylarını al
- `client.fine_tuning.jobs.cancel(<job_id>)` - Bir ince ayar işini iptal et
- `client.fine_tuning.jobs.list_events(fine_tuning_job_id=<job_id>, limit=<b>)` - İşten en fazla n olayı listele
- `client.fine_tuning.jobs.create(model="gpt-35-turbo", training_file="your-training-file.jsonl", ...)`

Sürecin ilk adımı, _eğitim dosyasını doğrulamaktır_ ve bu sayede verilerin doğru formatta olduğundan emin olunur.


In [26]:
from openai import OpenAI
client = OpenAI()

# List 10 fine-tuning jobs
client.fine_tuning.jobs.list(limit=10)

# Retrieve the state of a fine-tune
client.fine_tuning.jobs.retrieve(ft_filejob.id)

# List up to 10 events from a fine-tuning job
client.fine_tuning.jobs.list_events(fine_tuning_job_id=ft_filejob.id, limit=10)

SyncCursorPage[FineTuningJobEvent](data=[FineTuningJobEvent(id='ftevent-GkWiDgZmOsuv4q5cSTEGscY6', created_at=1715566184, level='info', message='Validating training file: file-JdAJcagdOTG6ACNlFWzuzmyV', object='fine_tuning.job.event', data={}, type='message'), FineTuningJobEvent(id='ftevent-3899xdVTO3LN7Q7LkKLMJUnb', created_at=1715566184, level='info', message='Created fine-tuning job: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', object='fine_tuning.job.event', data={}, type='message')], object='list', has_more=False)

In [30]:
# Once the training data is validated
# Track the job status to see if it is running and when it is complete
from openai import OpenAI
client = OpenAI()

response = client.fine_tuning.jobs.retrieve(ft_filejob.id)

print("Job ID:", response.id)
print("Status:", response.status)
print("Trained Tokens:", response.trained_tokens)

Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh
Status: running
Trained Tokens: None


### Adım 2.3: İlerlemenizi izlemek için olayları takip edin


In [44]:
# You can also track progress in a more granular way by checking for events
# Refresh this code till you get the `The job has successfully completed` message
response = client.fine_tuning.jobs.list_events(ft_filejob.id)

events = response.data
events.reverse()

for event in events:
    print(event.message)

Step 85/100: training loss=0.14
Step 86/100: training loss=0.00
Step 87/100: training loss=0.00
Step 88/100: training loss=0.07
Step 89/100: training loss=0.00
Step 90/100: training loss=0.00
Step 91/100: training loss=0.00
Step 92/100: training loss=0.00
Step 93/100: training loss=0.00
Step 94/100: training loss=0.00
Step 95/100: training loss=0.08
Step 96/100: training loss=0.05
Step 97/100: training loss=0.00
Step 98/100: training loss=0.00
Step 99/100: training loss=0.00
Step 100/100: training loss=0.00
Checkpoint created at step 80 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyyF2:ckpt-step-80
Checkpoint created at step 90 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyzhK:ckpt-step-90
New fine-tuned model created: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz
The job has successfully completed


### Adım 2.4: OpenAI Kontrol Panelinde durumu görüntüleyin


Ayrıca, OpenAI web sitesini ziyaret ederek ve platformun _Fine-tuning_ bölümünü inceleyerek durumu görüntüleyebilirsiniz. Bu bölümde mevcut işin durumunu görebilir ve önceki iş yürütme geçmişini takip edebilirsiniz. Bu ekran görüntüsünde, önceki çalıştırmanın başarısız olduğunu ve ikinci çalıştırmanın başarılı olduğunu görebilirsiniz. Bağlam açısından, bu durum ilk çalıştırmada yanlış biçimlendirilmiş kayıtlar içeren bir JSON dosyası kullanıldığında meydana geldi - düzeltildikten sonra, ikinci çalıştırma başarıyla tamamlandı ve modeli kullanıma sundu.

![Fine-tuning job status](../../../../../translated_images/fine-tuned-model-status.563271727bf7bfba7e3f73a201f8712fae3cea1c08f7c7f12ca469c06d234122.tr.png)


Durum mesajlarını ve metrikleri, aşağıda gösterildiği gibi görsel kontrol panelinde daha aşağıya kaydırarak da görüntüleyebilirsiniz:

| Mesajlar | Metrikler |
|:---|:---|
| ![Mesajlar](../../../../../translated_images/fine-tuned-messages-panel.4ed0c2da5ea1313b3a706a66f66bf5007c379cd9219cfb74cb30c0b04b90c4c8.tr.png) |  ![Metrikler](../../../../../translated_images/fine-tuned-metrics-panel.700d7e4995a652299584ab181536a6cfb67691a897a518b6c7a2aa0a17f1a30d.tr.png)|


### Adım 3.1: Kimliği Al & İnce Ayar Yapılmış Modeli Kodda Test Et


In [46]:
# Retrieve the identity of the fine-tuned model once ready
response = client.fine_tuning.jobs.retrieve(ft_filejob.id)
fine_tuned_model_id = response.fine_tuned_model
print("Fine-tuned Model ID:", fine_tuned_model_id)

Fine-tuned Model ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz


In [47]:
# You can then use that model to generate completions from the SDK as shown
# Or you can load that model into the OpenAI Playground (in the UI) to validate it from there.
from openai import OpenAI
client = OpenAI()

completion = client.chat.completions.create(
  model=fine_tuned_model_id,
  messages=[
    {"role": "system", "content": "You are Elle, a factual chatbot that answers questions about elements in the periodic table with a limerick"},
    {"role": "user", "content": "Tell me about Strontium"},
  ]
)
print(completion.choices[0].message)

ChatCompletionMessage(content="Strontium, a metal so bright - It's in fireworks, a dazzling sight - It's in bones, you see - And in tea, it's the key - It's the fortieth, so pure, that's the right", role='assistant', function_call=None, tool_calls=None)


### Adım 3.2: İncelenmiş Modeli Playground’da Yükleyin ve Test Edin

Artık ince ayar yapılmış modeli iki şekilde test edebilirsiniz. İlk olarak, Playground’a gidip Modeller açılır menüsünden yeni ince ayar yaptığınız modeli seçenekler arasından seçebilirsiniz. Diğer seçenek ise, İnce Ayar panelinde gösterilen "Playground" seçeneğini kullanmak (yukarıdaki ekran görüntüsüne bakın). Bu seçenek, temel ve ince ayar yapılmış model sürümlerini yan yana gösteren _karşılaştırmalı_ bir görünüm açar ve hızlıca değerlendirme yapmanızı sağlar.

Sistem bağlamını (context) eğitim verinizde kullandığınız şekilde doldurun ve test sorunuzla devam edin. Her iki tarafta da aynı bağlam ve soru ile güncelleme yapıldığını göreceksiniz. Karşılaştırmayı çalıştırın ve çıktılar arasındaki farkı gözlemleyin. _İnce ayar yapılmış modelin, örneklerinizde verdiğiniz formatta yanıt ürettiğine; temel modelin ise sadece sistem istemini takip ettiğine dikkat edin._

Karşılaştırma ekranında ayrıca her model için token sayısını ve çıkarım süresini görebilirsiniz. **Bu özel örnek, süreci göstermek için basit tutulmuştur ve gerçek bir veri setini veya senaryoyu yansıtmaz**. Her iki örnekte de token sayısının aynı olduğunu (sistem bağlamı ve kullanıcı istemi aynı) ve ince ayar yapılmış modelin çıkarım için daha fazla zaman harcadığını (özel model) fark edebilirsiniz.

Gerçek dünyada, bu tür basit bir örnek yerine gerçek verilerle (örneğin, müşteri hizmetleri için ürün kataloğu) ince ayar yapacaksınız ve yanıt kalitesi çok daha belirgin olacaktır. _Bu durumda, temel modelle eşdeğer yanıt kalitesi almak için daha fazla özel istem mühendisliği gerekecek, bu da token kullanımını ve muhtemelen çıkarım için gereken işlem süresini artıracaktır_. _Bunu denemek için, OpenAI Cookbook’taki ince ayar örneklerine göz atabilirsiniz._



---

**Feragatname**:  
Bu belge, AI çeviri hizmeti [Co-op Translator](https://github.com/Azure/co-op-translator) kullanılarak çevrilmiştir. Doğruluk için çaba göstersek de, otomatik çevirilerde hata veya yanlışlıklar bulunabileceğini lütfen unutmayın. Belgenin orijinal diliyle hazırlanmış hali esas alınmalıdır. Kritik bilgiler için profesyonel insan çevirisi önerilir. Bu çevirinin kullanımından doğabilecek yanlış anlama veya yanlış yorumlamalardan sorumlu değiliz.
